In [3]:
from astropy.table import Table
from dendrocat import RadioSource
from astropy.io import fits
from spectral_cube import SpectralCube
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import numpy as np
import sys
import importlib
from astropy.wcs import WCS
from scipy import interpolate
import pandas as pd
import sympy as syp
from astropy import units as u
import radio_beam
from astropy.table import Table
from radio_beam import Beam
from astropy import coordinates
from astropy import wcs
from astropy.nddata.utils import Cutout2D
from dendrocat.aperture import Ellipse
from astropy.io import ascii
import astropy.constants as c

from functools import reduce
from itertools import chain
import Paths.Paths as paths
from regions import Regions

Path = paths.filepaths()


class core_catdata():
    def __init__(self,catfile, almaimf_beam, hrfits, band='B3', recommend_criteria='b6'):
        catdata_w51e = ascii.read(catfile,data_start=0,format='commented_header', header_start=120,  comment="!")
        geo_avg_beam = np.sqrt(almaimf_beam.major.to(u.arcsec).value*almaimf_beam.minor.to(u.arcsec).value)
        rescaled_3mmflux = catdata_w51e['FXT_BST03'] * catdata_w51e['AFWHM02'] * catdata_w51e['BFWHM02'] / catdata_w51e['AFWHM03'] / catdata_w51e['BFWHM03']
        gamma = np.log10(catdata_w51e['FXT_BST02'] / rescaled_3mmflux)/np.log10(228.918/101.426)
        self.rec_ind_b6 = np.where((np.abs(catdata_w51e['GOODM02'])>1)&
                           (np.abs(catdata_w51e['SIGNM02'])>1)&
                           (catdata_w51e['FXP_BST02']/catdata_w51e['FXP_ERR02']>2)&
                           (catdata_w51e['FXT_BST02']/catdata_w51e['FXT_ERR02']>2)&
                           (catdata_w51e['AFWHM02']/catdata_w51e['BFWHM02']<2)&
                           (catdata_w51e['FOOA02']/catdata_w51e['AFWHM02']>1.15)&
                           (gamma>2))[0]
        self.rec_ind_b6b3 = np.where((np.abs(catdata_w51e['GOODM02'])>1)&
                           (np.abs(catdata_w51e['SIGNM02'])>1)&
                           (catdata_w51e['FXP_BST02']/catdata_w51e['FXP_ERR02']>2)&
                           (catdata_w51e['FXT_BST02']/catdata_w51e['FXT_ERR02']>2)&
                           (catdata_w51e['AFWHM02']/catdata_w51e['BFWHM02']<2)&
                           (catdata_w51e['FOOA02']/catdata_w51e['AFWHM02']>1.15)&
                           (np.sqrt(catdata_w51e['AFWHM03']*catdata_w51e['BFWHM03']) <4 * geo_avg_beam )&
                           (catdata_w51e['AFWHM03']/catdata_w51e['BFWHM03']<=2)&
                           (np.abs(catdata_w51e['SIGNM03'])>1)&
                           (catdata_w51e['FXP_BST03']/catdata_w51e['FXP_ERR03']>1)&
                           (catdata_w51e['FXT_BST03']/catdata_w51e['FXT_ERR03']>1)&
                           (gamma>2))[0]
        if recommend_criteria == 'b6':
            rec_ind_prep = self.rec_ind_b6
        elif recommend_criteria == 'b6b3':
            rec_ind_prep = self.rec_ind_b6b3
        yso_image = fits.open(hrfits)[0].data[0][0]
        hrhdr = fits.open(hrfits)[0].header
        inside_idx = exist_inside_FOV(catdata_w51e['WCS_ACOOR'], catdata_w51e['WCS_DCOOR'], yso_image, hrhdr)
        rec_ind = np.intersect1d(rec_ind_prep, inside_idx)
        print('total core number = ', len(catdata_w51e['WCS_ACOOR']))
        print(hrfits)
        print('robust %s measurement before applying FoV criteria = '%recommend_criteria, len(rec_ind_prep))
        print('robust %s measurement after applying FoV criteria = '%recommend_criteria, len(rec_ind))
        self.sky_ra = catdata_w51e['WCS_ACOOR'][rec_ind]
        self.sky_dec = catdata_w51e['WCS_DCOOR'][rec_ind]
        self.pix_x = catdata_w51e['XCO_P'][rec_ind]
        self.pix_y = catdata_w51e['YCO_P'][rec_ind]
        
        if band=='B3':
            bandind = 3
        else:
            bandind = 2

        self.afwhm = catdata_w51e['AFWHM0%d'%bandind][rec_ind] #arcsec
        self.bfwhm = catdata_w51e['BFWHM0%d'%bandind][rec_ind]

        self.asize = catdata_w51e['ASIZE0%d'%bandind][rec_ind]
        self.bsize = catdata_w51e['BSIZE0%d'%bandind][rec_ind]

        self.theta = catdata_w51e['THETA0%d'%bandind][rec_ind]
        
        self.flux = catdata_w51e['FXT_BST0%d'%bandind][rec_ind]
        self.fluxerr = catdata_w51e['FXT_ERR0%d'%bandind][rec_ind]
        self.flux_g = catdata_w51e['FXT_ALT0%d'%bandind][rec_ind]

        self.fooa = catdata_w51e['FOOA0%d'%bandind][rec_ind]
        self.foob = catdata_w51e['FOOB0%d'%bandind][rec_ind]
        
        self.int_peak = catdata_w51e['FXP_BST0%d'%bandind][rec_ind]
        self.interr_peak = catdata_w51e['FXP_ERR0%d'%bandind][rec_ind]
        
        self.size = (catdata_w51e['AFWHM0%d'%bandind][rec_ind]*catdata_w51e['BFWHM0%d'%bandind][rec_ind]**2)**(1/3)


        self.no = catdata_w51e['NO'][rec_ind]

        
        
        rescaled_3mmflux = catdata_w51e['FXT_BST03'][rec_ind] * catdata_w51e['AFWHM02'][rec_ind] * catdata_w51e['BFWHM02'][rec_ind] / catdata_w51e['AFWHM03'][rec_ind]/catdata_w51e['BFWHM03'][rec_ind]
        gamma = np.log10(catdata_w51e['FXT_BST02'][rec_ind] / rescaled_3mmflux)/np.log10(228.918/101.426)
        self.woffind = np.where(gamma>2)[0]
        self.wffind = np.where(gamma<2)[0]
        self.rec_ind = rec_ind

        
    def get_temperature(self,tempfile,threshold=2e-4):
        tab = ascii.read(tempfile, data_start=4)
        ra = tab['col2']
        dec = tab['col3']
        temp = tab['col4']

        temparr = []
        for i in range(len(self.sky_ra)):
            dist = np.sqrt((self.sky_ra[i]-ra)**2+(self.sky_dec[i]-dec)**2) #deg
            mindist = np.min(dist)
            if mindist<threshold:
                index = np.argmin(dist)
                temparr.append(temp[index])
            else:
                temparr.append(np.nan)
        self.temp = np.array(temparr)*u.K
        return np.array(temparr)
    
    def get_coremass(self,freq, kappa, dist):
       
        mass = - 1 / kappa * dist**2 * self.flux  / self.int_peak * np.log(1 - (self.int_peak *u.Jy  / BB(freq,self.temp)).to(u.cm/u.cm).value)
        self.mass = mass.to(u.Msun)
        return mass.to(u.Msun)
    
    def get_coremasserr(self):
        
        masserr = np.abs(self.mass * np.sqrt((self.fluxerr/self.flux)**2+(self.interr_peak/self.int_peak)**2))
        self.masserr = masserr
        return masserr.to(u.Msun)   
    def get_Jeansmass(self, dist=5410):
        rad = (self.afwhm.value*dist * self.bfwhm.value**2*dist**2)**(1/3) #arcsec -> au
        rho = self.mass / (4/3*np.pi * (rad*u.au)**3)
        c_s = np.sqrt(c.k_B*self.temp/2.8/c.m_p)
        jeans_mass = np.pi**(5/2)/6/c.G**(3/2) * c_s.to(u.cm/u.s)**3 * rho.to(u.g/u.cm**3)**(-1/2)
        
        return jeans_mass.to(u.Msun)

    def get_Jeansmasserr(self,dist=5410):
        rad = (self.afwhm.value*dist * self.bfwhm.value**2*dist**2)**(1/3) #arcsec -> au
        rho = self.mass / (4/3*np.pi * (rad*u.au)**3)
        c_s = np.sqrt(c.k_B*self.temp/2.8/c.m_p)
        jeans_mass = np.pi**(5/2)/6/c.G**(3/2) * c_s.to(u.cm/u.s)**3 * rho.to(u.g/u.cm**3)**(-1/2)
        rho_err = self.masserr / (4/3*np.pi * (rad*u.au)**3)

        jeans_mass_err = 0.5*jeans_mass * (rho_err/rho)

        return jeans_mass_err.to(u.Msun)
    def get_BE_mass(self, dist=5410):
        rad = (self.afwhm.value*dist * self.bfwhm.value**2*dist**2)**(1/3)
        rho = self.mass / (4/3*np.pi * (rad*u.au)**3)

        BE_mass = 1.18 * np.sqrt((c.k_B*self.temp/2.33/c.m_p/c.G)**3/rho).to(u.Msun)
        return BE_mass.to(u.Msun)
    
    def get_BE_masserr(self, dist=5410):
        rad = (self.afwhm.value*dist * self.bfwhm.value**2*dist**2)**(1/3)
        rho = self.mass / (4/3*np.pi * (rad*u.au)**3)

        BE_mass = 1.18 * np.sqrt((c.k_B*self.temp/2.33/c.m_p/c.G)**3/rho).to(u.Msun)
        rho_err = self.masserr / (4/3*np.pi * (rad*u.au)**3)

        BE_mass_err = 0.5*BE_mass * (rho_err/rho)
        return BE_mass_err.to(u.Msun)
    
    
    def multiplicity(self, hdrhr, xysource, almaimf_beam=None,issky=True, label=None,
                 aperture='fwhm+beam',limitarea=None,  maxrad=1e3, verbose=False):
        ncat = len(self.no)
        if limitarea is not None:
            posind = np.where((sky_ra>limitarea[0])&(sky_ra<limitarea[1])&
                              (sky_dec>limitarea[2])&(sky_dec<limitarea[3]))[0]
        else:
            posind = np.arange(ncat)
        if label is None:
            label=np.arange(ncat)
        coreidx = np.arange(ncat)

        totalind = np.intersect1d(posind, label)
        print('totalind',totalind)
        sky_ra = self.sky_ra[totalind]
        sky_dec = self.sky_dec[totalind]
        fooa = self.fooa[totalind]
        foob = self.foob[totalind]
        theta = self.theta[totalind]
        no = self.no[totalind]
        afwhm =self.afwhm[totalind]
        bfwhm = self.bfwhm[totalind]

        skypos = np.vstack((sky_ra,sky_dec)).T


        wcshr = WCS(hdrhr,naxis=2)
        scalehr = wcshr.proj_plane_pixel_scales()[0]

        xypos_core = wcshr.wcs_world2pix(skypos,0)
        xpos_core = xypos_core[:,0]
        ypos_core = xypos_core[:,1]

        if issky:
            xysource_pix = wcshr.wcs_world2pix(xysource,0)
        else:
            xysource_pix = xysource
        ysonumarr=[] ; coreindarr=[]; ysoindarr=[]
        prestellar_ind = []
        protostellar_ind = []
        #print(xysource)
        for i in range(len(xypos_core)):
            if aperture is 'footprint':
                major = fooa[i]/3600/scalehr.value
                minor = foob[i]/3600/scalehr.value
                pa = theta[i]+90
            elif aperture is 'fwhm+beam':
                meanbeamsize = (almaimf_beam.major.value+almaimf_beam.minor.value)/4
                major = (afwhm[i]/3600+meanbeamsize)/scalehr.value
                minor = (bfwhm[i]/3600+meanbeamsize)/scalehr.value
                pa = theta[i]+90
            elif aperture is 'beam':
                major = almaimf_beam.major.value/scalehr.value
                minor = almaimf_beam.minor.value/scalehr.value
                pa = almaimf_beam.pa.value+90
            #if major > maxrad:
            #    continue

            numsource, indsource = count_points_in_ellipse(xpos_core[i],ypos_core[i],major,minor,pa,xysource_pix[:,0],xysource_pix[:,1])
            print(totalind[i], indsource, numsource)
            ysonumarr.append(numsource)
            coreindarr.append(totalind[i])
            ysoindarr.append(indsource)
            if numsource==0:
                prestellar_ind.append(totalind[i])
            elif numsource>0:
                protostellar_ind.append(totalind[i])
            
            
      
        ysoindarr_flat = []
        for arr in ysoindarr:
            ysoindarr_flat.extend(arr)
       
        print('cores with/without YSOs ', len(np.where(np.array(ysonumarr)!=0)[0]), len(xpos_core)-len(np.where(np.array(ysonumarr)!=0)[0]))
        print('YSOs associated with cores',len(np.unique(ysoindarr_flat)),' out of ',len(xysource_pix[:,0]))
              
        self.ysonumarr = ysonumarr
        self.coreindarr = coreindarr
        self.ysoindarr = ysoindarr
        self.prestellar_ind = prestellar_ind
        self.protostellar_ind = protostellar_ind
        print('pre,pro',self.prestellar_ind, self.protostellar_ind)
        return ysonumarr, coreindarr, ysoindarr, prestellar_ind, protostellar_ind
        
        
    
        
        
    

                
            
def BB(freq, temp):
    B_nu = (2 * freq**3 *c.h / (c.c**2) * 1 / (np.e**(c.h*freq/(c.k_B*temp))-1))
    return B_nu        

def kappa(nu, nu0=271.1*u.GHz, kappa0=0.0114*u.cm**2*u.g**-1, beta=1.75):
    """
    Compute the opacity $\kappa$ given a reference frequency (or wavelength)
    and a power law governing the opacity as a fuction of frequency:
    $$ \kappa = \kappa_0 \left(\\frac{\\nu}{\\nu_0}\\right)^{\\beta} $$
    The default kappa=0.0114 at 271.1 GHz comes from extrapolating the
    Ossenkopf & Henning 1994 opacities for the thin-ice-mantle, 10^6 year model
    anchored at 1.0 mm with an assumed beta of 1.75.
    Parameters
    ----------
    nu: astropy.Quantity [u.spectral() equivalent]
        The frequency at which to evaluate kappa
    nu0: astropy.Quantity [u.spectral() equivalent]
        The reference frequency at which $\kappa$ is defined
    kappa0: astropy.Quantity [cm^2/g]
        The dust opacity per gram of H2 along the line of sight.  Because of
        the H2 conversion, this factor implicitly includes a dust to gas ratio
        (usually assumed 100)
    beta: float
        The power-law index governing kappa as a function of nu
    """
    return (kappa0*(nu.to(u.GHz,u.spectral())/nu0.to(u.GHz,u.spectral()))**(beta)).to(u.cm**2/u.g)        

     
"""
def get_BEmass(major, minor, temp ):
    rad = (major * minor**2)**(1/3)
    sigma_th = np.sqrt(c.k_B*temp*u.K/(2.4*c.m_p))
    return (2.4*rad*u.au*sigma_th**2/c.G).to(u.Msun)


"""
def count_points_in_ellipse(xcen,ycen,major,minor,pa,xpos,ypos):
    # caclulate foci positions 
    # must be full major, minor not semi major, minor
    fodx = np.cos(pa)*np.sqrt((major/2)**2-(minor/2)**2)
    fody = np.sin(pa)*np.sqrt((major/2)**2-(minor/2)**2)
    
    fox1 = xcen + fodx
    foy1 = ycen + fody
    
    fox2 = xcen - fodx
    foy2 = ycen - fody
    
    dist1 = np.sqrt((xpos-fox1)**2+(ypos-foy1)**2)
    dist2 = np.sqrt((xpos-fox2)**2+(ypos-foy2)**2)
    
    isinside = np.where(dist1+dist2<major)[0]
    
    return len(isinside), list(isinside)



def exist_inside_FOV(xcore_sky, ycore_sky, yso_image, hrhdr):
    isfinite = np.where(np.isfinite(yso_image))
    xfinite1 = np.min(isfinite[0])
    xfinite2 = np.max(isfinite[0])
    yfinite1 = np.min(isfinite[1])
    yfinite2 = np.max(isfinite[1])
    
    xcen = (xfinite1 + xfinite2)/2
    ycen = (yfinite1 + yfinite2)/2

    wcs_hr = WCS(hrhdr,naxis=2)
    xypos = wcs_hr.wcs_world2pix(np.vstack((xcore_sky,ycore_sky)).T,0)
    
    rad = np.abs(xfinite2 - xcen)
    
    
    dist = np.sqrt((xypos[:,0]- xcen)**2+(xypos[:,1]-ycen)**2)
    isinside = np.where(dist<rad)[0]
    
    return isinside
     

In [8]:
from regions import EllipseSkyRegion
from astropy.coordinates import SkyCoord

fitsdata_b3 = fits.open(Path.w51e_b3_tt0)
image_b3_w51e = fitsdata_b3[0].data
if len(image_b3_w51e.shape)>2:
    image_b3_w51e = fitsdata_b3[0].data[0][0]
hdrB3_w51e_hr = fits.getheader(Path.w51e_b3_tt0)  
    
fitsdata_b3 = fits.open(Path.w51n_b3_tt0)
image_b3_w51n = fitsdata_b3[0].data
if len(image_b3_w51n.shape)>2:
    image_b3_w51n = fitsdata_b3[0].data[0][0]
hdrB3_w51n_hr = fits.getheader(Path.w51n_b3_tt0)  

fitsdata_b6 = fits.open(Path.w51e_b6_cont)
image_b6_w51e = fitsdata_b6[0].data
if len(image_b6_w51e.shape)>2:
    image_b6_w51e = fitsdata_b6[0].data[0][0]
hdrB6_w51e_hr = fits.getheader(Path.w51e_b6_cont)  
    
fitsdata_b6 = fits.open(Path.w51n_b6_cont)
image_b6_w51n = fitsdata_b6[0].data
if len(image_b6_w51n.shape)>2:
    image_b6_w51n = fitsdata_b6[0].data[0][0]
hdrB6_w51n_hr = fits.getheader(Path.w51n_b6_cont)  

hdrB6_w51e_almaimf = fits.getheader(Path.w51e_b6_almaimf)  
wcsB6_w51e_almaimf = WCS(hdrB6_w51e_almaimf,naxis=2)
beamb6_w51e_almaimf = Beam.from_fits_header(hdrB6_w51e_almaimf)  

hdrB3_w51e_almaimf = fits.getheader(Path.w51e_b3_almaimf)  
wcsB3_w51e_almaimf = WCS(hdrB3_w51e_almaimf,naxis=2)
beamb3_w51e_almaimf = Beam.from_fits_header(hdrB3_w51e_almaimf) 

hdrB6_w51n_almaimf = fits.getheader(Path.w51n_b6_almaimf)  
wcsB6_w51n_almaimf = WCS(hdrB6_w51n_almaimf,naxis=2)
beamb6_w51n_almaimf = Beam.from_fits_header(hdrB6_w51n_almaimf)  

hdrB3_w51n_almaimf = fits.getheader(Path.w51n_b3_almaimf)  
wcsB3_w51n_almaimf = WCS(hdrB3_w51n_almaimf,naxis=2)
beamb3_w51n_almaimf = Beam.from_fits_header(hdrB3_w51n_almaimf)  

freqb3=92982346121.91989*u.Hz
freqb6=226691598706.70853*u.Hz
kappa_b3 = kappa(freqb3)
kappa_b6 = kappa(freqb6)



core_catdata_w51e_b6_b6crit_b3fov = core_catdata(Path.w51e_almaimf_catalog,beamb3_w51e_almaimf, Path.w51e_b3_tt0,band='B6',)
core_catdata_w51e_b6_b6crit_b6fov = core_catdata(Path.w51e_almaimf_catalog,beamb3_w51e_almaimf, Path.w51e_b6_cont, band='B6')


core_catdata_w51n_b6_b6crit_b3fov = core_catdata(Path.w51n_almaimf_catalog, beamb3_w51n_almaimf,Path.w51n_b3_tt0, band='B6')
core_catdata_w51n_b6_b6crit_b6fov = core_catdata(Path.w51n_almaimf_catalog, beamb3_w51n_almaimf,Path.w51n_b6_cont, band='B6')



sky_ra_w51e = core_catdata_w51e_b6_b6crit_b3fov.sky_ra
sky_dec_w51e = core_catdata_w51e_b6_b6crit_b3fov.sky_dec
major_w51e = core_catdata_w51e_b6_b6crit_b3fov.afwhm
minor_w51e = core_catdata_w51e_b6_b6crit_b3fov.bfwhm
thetas_w51e = core_catdata_w51e_b6_b6crit_b3fov.theta
sky_ra_w51n = core_catdata_w51n_b6_b6crit_b3fov.sky_ra
sky_dec_w51n = core_catdata_w51n_b6_b6crit_b3fov.sky_dec
major_w51n = core_catdata_w51n_b6_b6crit_b3fov.afwhm
minor_w51n = core_catdata_w51n_b6_b6crit_b3fov.bfwhm
thetas_w51n = core_catdata_w51n_b6_b6crit_b3fov.theta

regions_w51e = Regions([EllipseSkyRegion(center=SkyCoord(ra, dec, unit='deg'), 
                            width=afwhm*u.arcsec,
                            height=bfwhm*u.arcsec,
                            angle=90*u.deg+theta*u.deg)
           for ra,dec,afwhm,bfwhm,theta in zip(sky_ra_w51e,
                                               sky_dec_w51e,
                                               major_w51e,
                                               minor_w51e,
                                               thetas_w51e
                                               )])
regions_w51n = Regions([EllipseSkyRegion(center=SkyCoord(ra, dec, unit='deg'), 
                            width=afwhm*u.arcsec,
                            height=bfwhm*u.arcsec,
                            angle=90*u.deg+theta*u.deg)
           for ra,dec,afwhm,bfwhm,theta in zip(sky_ra_w51n,
                                               sky_dec_w51n,
                                               major_w51n,
                                               minor_w51n,
                                               thetas_w51n
                                               )])
regions_w51e.write('/orange/adamginsburg/w51/TaehwaYoo/almaimf_core_w51e.reg', format='ds9',overwrite=True)
regions_w51n.write('/orange/adamginsburg/w51/TaehwaYoo/almaimf_core_w51n.reg', format='ds9',overwrite=True)
regions_w51e.write('/orange/adamginsburg/w51/TaehwaYoo/almaimf_core_w51e_reg.fits', format='fits',overwrite=True)
regions_w51n.write('/orange/adamginsburg/w51/TaehwaYoo/almaimf_core_w51n_reg.fits', format='fits',overwrite=True)
regions_w51e.write('/orange/adamginsburg/w51/TaehwaYoo/almaimf_core_w51e_reg.crtf', format='crtf',overwrite=True)
regions_w51n.write('/orange/adamginsburg/w51/TaehwaYoo/almaimf_core_w51n_reg.crtf', format='crtf',overwrite=True)

total core number =  41
/orange/adamginsburg/w51/TaehwaYoo/2017.1.00293.S_W51_B3_LB/may2021_successful_imaging/w51e2.spw0thru19.14500.robust0.thr0.075mJy.mfs.I.startmod.selfcal7.image.tt0.pbcor.fits
robust b6 measurement before applying FoV criteria =  18
robust b6 measurement after applying FoV criteria =  18
total core number =  41
/orange/adamginsburg/w51/TaehwaYoo/w51e2.spw0thru19.14500.robust0.thr0.15mJy.mfs.I.startmod.selfcal7.image.tt0.pbcor.fits
robust b6 measurement before applying FoV criteria =  18
robust b6 measurement after applying FoV criteria =  10
total core number =  127
/orange/adamginsburg/w51/TaehwaYoo/2017.1.00293.S_W51_B3_LB/may2021_successful_imaging/w51n.spw0thru19.14500.robust0.thr0.075mJy.mfs.I.startmod.selfcal7.image.tt0.pbcor.fits
robust b6 measurement before applying FoV criteria =  72
robust b6 measurement after applying FoV criteria =  61
total core number =  127
/orange/adamginsburg/w51/TaehwaYoo/w51n.spw0thru19.14500.robust0.thr0.1mJy.mfs.I.startmod.se

In [2]:
catdata_w51e_b6 = catdata(cat_w51e, band="B3")
sky_ra_w51e_b6 = catdata_w51e_b6.sky_ra
sky_dec_w51e_b6 = catdata_w51e_b6.sky_dec
major_w51e = catdata_w51e_b6.afwhm
minor_w51e = catdata_w51e_b6.bfwhm
thetas_w51e = catdata_w51e_b6.theta
ind_w51e = catdata_w51e_b6.rec_ind_b6

catdata_w51n_b6 = catdata(cat_w51n, band="B3")
sky_ra_w51n_b6 = catdata_w51n_b6.sky_ra
sky_dec_w51n_b6 = catdata_w51n_b6.sky_dec
major_w51n = catdata_w51n_b6.afwhm
minor_w51n = catdata_w51n_b6.bfwhm
thetas_w51n = catdata_w51n_b6.theta
ind_w51n = catdata_w51n_b6.rec_ind_b6

isinside_w51e_b3 = exist_inside_FOV(sky_ra_w51e_b6, sky_dec_w51e_b6, image_b3_w51e, hdrB3_w51e_hr)
isinside_w51n_b3 = exist_inside_FOV(sky_ra_w51n_b6, sky_dec_w51n_b6, image_b3_w51n, hdrB3_w51n_hr)

totalind_w51e = reduce(np.intersect1d, (ind_w51e,isinside_w51e_b3 ))  
totalind_w51n = reduce(np.intersect1d, (ind_w51n,isinside_w51n_b3 ))  



regions_w51e = Regions([EllipseSkyRegion(center=SkyCoord(ra, dec, unit='deg'), 
                            width=afwhm*u.arcsec,
                            height=bfwhm*u.arcsec,
                            angle=90*u.deg+theta*u.deg)
           for ra,dec,afwhm,bfwhm,theta in zip(sky_ra_w51e_b6[totalind_w51e],
                                               sky_dec_w51e_b6[totalind_w51e],
                                               major_w51e[totalind_w51e],
                                               minor_w51e[totalind_w51e],
                                               thetas_w51e[totalind_w51e]
                                               )])
regions_w51n = Regions([EllipseSkyRegion(center=SkyCoord(ra, dec, unit='deg'), 
                            width=afwhm*u.arcsec,
                            height=bfwhm*u.arcsec,
                            angle=90*u.deg+theta*u.deg)
           for ra,dec,afwhm,bfwhm,theta in zip(sky_ra_w51n_b6[totalind_w51n],
                                               sky_dec_w51n_b6[totalind_w51n],
                                               major_w51n[totalind_w51n],
                                               minor_w51n[totalind_w51n],
                                               thetas_w51n[totalind_w51n]
                                               )])
regions_w51e.write('/orange/adamginsburg/w51/TaehwaYoo/almaimf_core_w51e_b3.reg', format='ds9')
regions_w51n.write('/orange/adamginsburg/w51/TaehwaYoo/almaimf_core_w51n_b3.reg', format='ds9')
